# Power electricity power consumption prediction model

Let's first start by importing the needed libraries

In [1]:
from pyspark.sql import SparkSession

And create a local parallel spark session

In [2]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName("Power electricity prediction") \
        .getOrCreate()

The we read the dataframe we need to apply the prediction on

In [3]:
train = spark.read.csv('../data/engineered/powerelectricity_train.csv', header=True, inferSchema=True)
test = spark.read.csv('../data/engineered/powerelectricity_test.csv', header=True, inferSchema=True)

Now we will test some regression models and evaluate them using rmse

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler

In [5]:
train_ep = train.drop('rms_current')
train_ep = train.drop('electric_power')

#train_rms = train.drop('electric_power')

test_ep = test.drop('rms_current')
test_ep = test.drop('electric_power')

#test_rms = test.drop('electric_power')

features_ep = train_ep.columns.copy()
#features_ep.remove('electric_power')
features_ep.remove('delta_y')

#features_rms = train_rms.columns.copy()
#features_rms.remove('rms_current')

In [6]:
assembler_pe_train = VectorAssembler(inputCols=features_ep,outputCol="features")
train_ep = assembler_pe_train.transform(train_ep)

Let's start by predicting electric_power

In [7]:
rf_pe = RandomForestRegressor(featuresCol="features",labelCol='delta_y',numTrees=12, maxDepth=12,seed=42)
#rf_pe = GBTRegressor(featuresCol="features",labelCol='electric_power',seed=42)
#rf_pe = LinearRegression(featuresCol="features",labelCol='electric_power',maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [8]:
%%time

model_pe = rf_pe.fit(train_ep)

CPU times: user 15.6 ms, sys: 10.9 ms, total: 26.5 ms
Wall time: 17.3 s


In [9]:
assembler_pe_test = VectorAssembler(inputCols=features_ep,outputCol="features")
test_ep = assembler_pe_test.transform(test_ep)

In [10]:
predictions = model_pe.transform(test_ep)

In [11]:
evaluator_rmse = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="rmse")
rmse = evaluator_rmse.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

evaluator_r2 = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="r2")
r2 = evaluator_r2.evaluate(predictions)
print("R squared (r2) on test data = %g" % r2)

evaluator_mae = RegressionEvaluator(
    labelCol="delta_y", predictionCol="prediction", metricName="mae")
mae = evaluator_mae.evaluate(predictions)
print("Mean Average Error (RMSE) on test data = %g" % mae)

Root Mean Squared Error (RMSE) on test data = 0.504772
R squared (r2) on test data = 0.965193
Mean Average Error (RMSE) on test data = 0.350798


In [15]:
predictions.select(["delta_y", "prediction"]).show(100000)

+--------------------+--------------------+
|             delta_y|          prediction|
+--------------------+--------------------+
| 0.12460495845226838| 0.08539921829750913|
| 0.12460495845226838| 0.08539921829750913|
| 0.12460495845226838|   0.079297496051295|
|-0.00269709431067...|0.012543368947219127|
|-0.00269709431067...|0.010590531165293253|
| 0.07497739445720697|0.010590531165293253|
| 0.17496202681168427|   0.079297496051295|
| 0.16960881381023007|   0.079297496051295|
| 0.26217378803479185| 0.07739010317301855|
|  0.1984058676154845| 0.07739010317301855|
|-0.43652422897623744|-0.33721374157729755|
| -0.4865442068489436|-0.41426276206231427|
| 0.24874197983736934|  0.0856488456337108|
| 0.24874197983736934| 0.07739010317301855|
|-0.00241365627515...|0.012543368947219127|
| -0.2339234821195113|-0.06126342618600663|
| -0.2771871301623623|-0.17085887226603733|
|-0.31066977079648694|-0.21749795853152443|
| -0.6151248703365599| -0.6589572695583682|
| 0.36482706462117775|   0.21310